# Задача

На основании коллекции размеченных коротких сообщений построить модель, предсказывающую общую тональность сообщения. Множество меток в данном случае - {позитивное, нейтральное, негативное}.

# Методы решения

Задачи сентимент-анализа решаются с помощью следующих подходов:  
    - подходы, основанные на правилах;  
    - подходы, основанные на словарях;  
    - машинное обучение с учителем;  
    - машинное обучение без учителя.


# NB

- Классификация выполняется легко и быстро  
- Хорошо работает с категорийными признаками  
- Приходится делать допущение о независимости признаков

# SVM

- Эффективен в случаях где набор измерений больше, чем обучающих данных
- Эффективен при многомерных векторах
- Небольшие изменения в обучающих данных не очень влияют на работу свм
- Не подходит для огромных датасетов
- Не очень справляется с шумом
- Медленнее по времени

# Libraries

In [2]:
import pandas as pd
import numpy as np
import re
import codecs
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus.reader import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# Data preparation

In [3]:
def load_data(path, header):
    marks_df = pd.read_csv(path, header=header)
    return marks_df

np.random.seed(500)

data = pd.read_csv('train.txt', encoding='utf-8', delimiter = '\t', names=['label', 'text'])
test = pd.read_csv('test_1.txt', encoding='utf-8', delimiter = '\t', names=['label', 'text'])
    
data['text'] = [re.sub(r'[^\w\s]', '', entry) for entry in data['text']]
test['text'] = [re.sub(r'[^\w\s]', '', entry) for entry in test['text']]

# Lower case
data['text'] = [entry.lower() for entry in data['text']]
test['text'] = [entry.lower() for entry in test['text']]

# Tokenization
data['text'] = [word_tokenize(entry) for entry in data['text']]
test['text'] = [word_tokenize(entry) for entry in test['text']]

# Removing Stop Words
russian_stopwords = set(stopwords.words('russian'))
russian_stopwords -= {'хорошо', 'ничего', 'нет', 'не', 'никогда', 'нельзя', 'ни', 'лучше', 'другой', 'впрочем', 'опять', 'более'}
for i, s in enumerate(data['text']):
    data['text'][i] =  [entry for entry in data['text'][i] if entry not in russian_stopwords]
for i, s in enumerate(test['text']):
    test['text'][i] =  [entry for entry in test['text'][i] if entry not in russian_stopwords]

# Lemmatizing
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    res = list()
    for word in text:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

data['text'] = [lemmatize(entry) for entry in data['text']]
test['text'] = [lemmatize(entry) for entry in test['text']]

Train_X, Train_Y, Test_X, Test_Y = [' '.join(s) for s in data['text']], data['label'], [' '.join(s) for s in test['text']], test['label']
encoder = {'__label__neg': 0, '__label__neut': 1, '__label__pos': 2}
decoder = ['__label__neg', '__label__neut', '__label__pos']
for i, label in enumerate(Train_Y):
    Train_Y[i] = encoder[label]
for i, label in enumerate(Test_Y):
    Test_Y[i] = encoder[label]

# TF-IDF

In [4]:
tfidf = TfidfVectorizer(ngram_range=(1,2))
to_fit = []
for s in data['text']:
    to_fit.append(' '.join(s))
for s in test['text']:
    to_fit.append(' '.join(s))
tfidf.fit(to_fit)
Train_X_tfidf = tfidf.transform(Train_X)
Test_X_tfidf = tfidf.transform(Test_X)

# SVM

In [13]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
Train_Y = Train_Y.astype('int')
SVM.fit(Train_X_tfidf, Train_Y)
predictions_SVM = SVM.predict(Test_X_tfidf)
Test_Y = Test_Y.astype('int')
print(predictions_SVM[:200])
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y)*100)

[0 0 2 0 2 1 0 0 0 1 0 2 0 2 1 1 0 1 2 1 2 0 0 1 0 0 2 2 1 2 1 1 1 1 1 2 2
 0 1 2 2 0 2 2 1 2 2 0 0 2 1 1 1 1 2 2 0 2 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1
 2 0 0 2 1 2 1 1 1 0 1 0 1 1 1 1 1 1 1 2 2 1 2 1 0 0 0 1 0 0 1 1 1 1 1 1 0
 0 1 1 0 0 2 0 0 1 0 1 0 2 1 1 0 0 1 0 1 0 1 1 1 2 0 0 1 0 0 0 2 1 2 2 1 1
 0 1 2 1 2 2 0 2 0 0 0 1 1 1 1 1 1 1 0 1 2 0 0 2 0 0 0 1 1 2 0 1 1 1 0 1 2
 0 1 0 2 0 0 2 2 0 0 0 0 2 2 0]
SVM Accuracy Score ->  76.2582056892779


# NB

In [6]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_tfidf)
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y)*100)


Naive Bayes Accuracy Score ->  74.9890590809628


In [7]:
report = classification_report(Test_Y, predictions_NB, digits=3)
print('NB\n', report)
report = classification_report(Test_Y, predictions_SVM, digits=3)
print('SVM\n', report)

NB
               precision    recall  f1-score   support

           0      0.756     0.799     0.777      1614
           1      0.728     0.741     0.735      1546
           2      0.768     0.703     0.734      1410

    accuracy                          0.750      4570
   macro avg      0.751     0.748     0.749      4570
weighted avg      0.750     0.750     0.749      4570

SVM
               precision    recall  f1-score   support

           0      0.763     0.803     0.782      1614
           1      0.728     0.766     0.746      1546
           2      0.808     0.713     0.757      1410

    accuracy                          0.763      4570
   macro avg      0.766     0.761     0.762      4570
weighted avg      0.765     0.763     0.762      4570



In [17]:
fin = codecs.open('check_tweets.txt', 'r', 'utf-8')
tweets_to_predict = fin.readlines()
tweets_to_predict = [tweet.rstrip() for tweet in tweets_to_predict]
started_tweets = tweets_to_predict

tweets_to_predict = [re.sub(r'[^\w\s]', '', entry) for entry in tweets_to_predict]
# Lower case
tweets_to_predict = [entry.lower() for entry in tweets_to_predict]
# Tokenization
tweets_to_predict = [word_tokenize(entry) for entry in tweets_to_predict]
# Removing Stop Words
for i, s in enumerate(tweets_to_predict):
    tweets_to_predict[i] =  [entry for entry in tweets_to_predict[i] if entry not in russian_stopwords]

morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    res = list()
    for word in text:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
    return res

tweets_to_predict = [lemmatize(entry) for entry in tweets_to_predict]

tweets_to_predict = [' '.join(s) for s in tweets_to_predict]

vectorized_tweets = tfidf.transform(tweets_to_predict)

predicted = SVM.predict(vectorized_tweets)
for i, label in enumerate(predicted):
    print(decoder[label], started_tweets[i])

__label__neg Мдааа а что вы еще ожидали от мошенников
__label__pos Люблю Россию сильной любовью..........
__label__neg я не счастлив и мне это не нравится!
__label__neg Путин любит быдло
__label__neg никому порекомендовать!!! этот БРЕД не могу
__label__neg Крым
__label__pos птички поют, на душе хорошо!
__label__pos этот фильм ужасов супер, противный как надо
__label__pos я поспала здоровым 4-часовым сном
__label__neut Поскольку ТКС Банк работает со своими клиентами исключительно через дистанционные каналы, центр обслуживания клиентов является одним из основных каналов коммуникации с клиентами.
__label__pos Зенит выиграл у Спартака
__label__neut Москва
__label__neut Комсомольск-на-Амуре
__label__neut Санкт-Петербург
